In [1]:
import os
import pandas as pd
import glob
import geopandas as gpd
import datetime
import numpy as np
import csv
import copy
import json

# 1 Define the Hyperparameters

In [2]:
maximum_affected_upstream_link_number = 10
maximum_affected_mile = 1
start_analysis_timestamp = datetime.datetime.fromisoformat('2020-01-01T00:00:00-05:00').timestamp()
end_analysis_timestamp = datetime.datetime.fromisoformat('2020-02-28T23:55:00-05:00').timestamp()
previous_affect_time = 30 # in minute
after_affect_time = 60 # in minute

In [38]:
federal_holiday_list = [datetime.date(2020, 1, 1), datetime.date(2020, 1, 20), datetime.date(2020, 2, 17)]

# 2 Process Incident
## 2.1 Load Original Incident File

In [4]:

Incident_Dataframe = pd.read_csv(os.getcwd()+'/Incidents/events_result_loop_keep_45_subset.csv')

In [5]:
Incident_Dataframe

,Event_ID,Event_Type,Start_Time,Closed_Time,Latitude,Longitude,Road,Direction,Max_Lanes_Closed,A,B,LINKID
0,DDOT_CAPTOP_129937,1 Vehicle Collision,2020-01-04T15:48:44-05:00,2020-01-04T16:33:44-05:00,38.894714,-76.952886,DC 295,North,1.0,20991,20990,23731
1,DDOT_CAPTOP_130046,2 Vehicle Collision,2020-01-11T00:43:46-05:00,2020-01-11T01:28:46-05:00,38.823044,-77.016892,I-295,South,NaN,20942,20943,56318
2,DDOT_CAPTOP_130282,3+ Vehicle Collision,2020-01-28T06:46:50-05:00,2020-01-28T07:08:11-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943
3,DDOT_CAPTOP_130444,2 Vehicle Collision,2020-02-07T08:44:26-05:00,2020-02-07T13:50:00-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943
4,DDOT_CAPTOP_130627,2 Vehicle Collision,2020-02-19T23:59:24-05:00,2020-02-20T00:15:00-05:00,38.894714,-76.952886,DC 295,North,NaN,20991,20990,23731
...,...,...,...,...,...,...,...,...,...,...,...,...
12135,VDOT_INNO2387556489-03202020,Vehicle Fire,2020-03-20T14:52:26-04:00,2020-03-20T17:03:00-04:00,38.757604,-77.162584,RT 613,North,2.0,36383,36384,5095
12136,VDOT_INNO2397812971-03232020,Vehicle Fire,2020-03-23T13:57:47-04:00,2020-03-23T14:15:54-04:00,38.426852,-77.407128,JEFFERSON DAVIS HWY,North,3.0,51245,51257,23146
12137,HoCoCAD_P200540235,Vehicle Fire,2020-02-23T16:27:46-05:00,2020-02-23T16:36:36-05:00,39.230550,-76.975340,NaN,East,NaN,44211,44210,8310
12138,HoCoCAD_F20006174,Vehicle Fire,2020-02-23T16:03:38-05:00,2020-02-23T16:34:36-05:00,39.163630,-76.791870,CRAZY RAYS PREVIOUS BUSINESS NAME,South,NaN,44274,44019,52465


## 2.2 Filter Incident by Analysis Date

In [6]:
start_timestamp_list = [datetime.datetime.fromisoformat(start_time).timestamp() for start_time in Incident_Dataframe['Start_Time'].to_list()]
Incident_Dataframe['start_timestamp'] = start_timestamp_list
close_timestamp_list = [datetime.datetime.fromisoformat(close_time).timestamp() for close_time in Incident_Dataframe['Closed_Time'].to_list()]
Incident_Dataframe['close_timestamp'] = close_timestamp_list

In [7]:
Incident_Dataframe_JanFeb = Incident_Dataframe[(Incident_Dataframe['start_timestamp']>start_analysis_timestamp)&(Incident_Dataframe['close_timestamp']<end_analysis_timestamp)]

In [8]:
Incident_Dataframe_JanFeb

,Event_ID,Event_Type,Start_Time,Closed_Time,Latitude,Longitude,Road,Direction,Max_Lanes_Closed,A,B,LINKID,start_timestamp,close_timestamp
0,DDOT_CAPTOP_129937,1 Vehicle Collision,2020-01-04T15:48:44-05:00,2020-01-04T16:33:44-05:00,38.894714,-76.952886,DC 295,North,1.0,20991,20990,23731,1.578171e+09,1.578174e+09
1,DDOT_CAPTOP_130046,2 Vehicle Collision,2020-01-11T00:43:46-05:00,2020-01-11T01:28:46-05:00,38.823044,-77.016892,I-295,South,NaN,20942,20943,56318,1.578721e+09,1.578724e+09
2,DDOT_CAPTOP_130282,3+ Vehicle Collision,2020-01-28T06:46:50-05:00,2020-01-28T07:08:11-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.580212e+09,1.580213e+09
3,DDOT_CAPTOP_130444,2 Vehicle Collision,2020-02-07T08:44:26-05:00,2020-02-07T13:50:00-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.581083e+09,1.581101e+09
4,DDOT_CAPTOP_130627,2 Vehicle Collision,2020-02-19T23:59:24-05:00,2020-02-20T00:15:00-05:00,38.894714,-76.952886,DC 295,North,NaN,20991,20990,23731,1.582175e+09,1.582176e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129,HoCoCAD_F20003865,Vehicle Fire,2020-02-03T10:48:40-05:00,2020-02-03T11:02:16-05:00,39.220850,-76.789000,NaN,South,NaN,44254,44017,17124,1.580745e+09,1.580746e+09
12130,HoCoCAD_P200340147,Vehicle Fire,2020-02-03T10:48:40-05:00,2020-02-03T11:02:16-05:00,39.220850,-76.789000,NaN,South,NaN,44254,44017,17124,1.580745e+09,1.580746e+09
12137,HoCoCAD_P200540235,Vehicle Fire,2020-02-23T16:27:46-05:00,2020-02-23T16:36:36-05:00,39.230550,-76.975340,NaN,East,NaN,44211,44210,8310,1.582493e+09,1.582494e+09
12138,HoCoCAD_F20006174,Vehicle Fire,2020-02-23T16:03:38-05:00,2020-02-23T16:34:36-05:00,39.163630,-76.791870,CRAZY RAYS PREVIOUS BUSINESS NAME,South,NaN,44274,44019,52465,1.582492e+09,1.582494e+09


## 2.2 Compute the Start and End Infulence Time of each Incident

In [9]:
def s_infulence_time(start_time):
    if start_analysis_timestamp > start_time - previous_affect_time*60:
        start_infulence_time = start_analysis_timestamp
    else:
        start_infulence_time = start_time - previous_affect_time*60
    return start_infulence_time

In [10]:
def e_infulence_time(close_time):
    if end_analysis_timestamp < close_time + after_affect_time*60:
        end_infulence_time = end_analysis_timestamp
    else:
        end_infulence_time = close_time + after_affect_time*60
    return end_infulence_time

In [11]:
Incident_Dataframe_JanFeb['s_infulence_time']=[s_infulence_time(time) for time in Incident_Dataframe_JanFeb['start_timestamp'].to_list()]
Incident_Dataframe_JanFeb['e_infulence_time']=[e_infulence_time(time) for time in Incident_Dataframe_JanFeb['close_timestamp'].to_list()]

/tmp/ipykernel_29949/2271188518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb['s_infulence_time']=[s_infulence_time(time) for time in Incident_Dataframe_JanFeb['start_timestamp'].to_list()]
/tmp/ipykernel_29949/2271188518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb['e_infulence_time']=[e_infulence_time(time) for time in Incident_Dataframe_JanFeb['close_timestamp'].to_list()]


In [12]:
Incident_Dataframe_JanFeb

,Event_ID,Event_Type,Start_Time,Closed_Time,Latitude,Longitude,Road,Direction,Max_Lanes_Closed,A,B,LINKID,start_timestamp,close_timestamp,s_infulence_time,e_infulence_time
0,DDOT_CAPTOP_129937,1 Vehicle Collision,2020-01-04T15:48:44-05:00,2020-01-04T16:33:44-05:00,38.894714,-76.952886,DC 295,North,1.0,20991,20990,23731,1.578171e+09,1.578174e+09,1.578169e+09,1.578177e+09
1,DDOT_CAPTOP_130046,2 Vehicle Collision,2020-01-11T00:43:46-05:00,2020-01-11T01:28:46-05:00,38.823044,-77.016892,I-295,South,NaN,20942,20943,56318,1.578721e+09,1.578724e+09,1.578720e+09,1.578728e+09
2,DDOT_CAPTOP_130282,3+ Vehicle Collision,2020-01-28T06:46:50-05:00,2020-01-28T07:08:11-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.580212e+09,1.580213e+09,1.580210e+09,1.580217e+09
3,DDOT_CAPTOP_130444,2 Vehicle Collision,2020-02-07T08:44:26-05:00,2020-02-07T13:50:00-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.581083e+09,1.581101e+09,1.581081e+09,1.581105e+09
4,DDOT_CAPTOP_130627,2 Vehicle Collision,2020-02-19T23:59:24-05:00,2020-02-20T00:15:00-05:00,38.894714,-76.952886,DC 295,North,NaN,20991,20990,23731,1.582175e+09,1.582176e+09,1.582173e+09,1.582179e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129,HoCoCAD_F20003865,Vehicle Fire,2020-02-03T10:48:40-05:00,2020-02-03T11:02:16-05:00,39.220850,-76.789000,NaN,South,NaN,44254,44017,17124,1.580745e+09,1.580746e+09,1.580743e+09,1.580749e+09
12130,HoCoCAD_P200340147,Vehicle Fire,2020-02-03T10:48:40-05:00,2020-02-03T11:02:16-05:00,39.220850,-76.789000,NaN,South,NaN,44254,44017,17124,1.580745e+09,1.580746e+09,1.580743e+09,1.580749e+09
12137,HoCoCAD_P200540235,Vehicle Fire,2020-02-23T16:27:46-05:00,2020-02-23T16:36:36-05:00,39.230550,-76.975340,NaN,East,NaN,44211,44210,8310,1.582493e+09,1.582494e+09,1.582491e+09,1.582497e+09
12138,HoCoCAD_F20006174,Vehicle Fire,2020-02-23T16:03:38-05:00,2020-02-23T16:34:36-05:00,39.163630,-76.791870,CRAZY RAYS PREVIOUS BUSINESS NAME,South,NaN,44274,44019,52465,1.582492e+09,1.582494e+09,1.582490e+09,1.582497e+09


# 3 Load Geo Info
## 3.1 Load and Process Consolidated Network Data

In [13]:
link_file = gpd.read_file(os.getcwd()+'/Network_consolidated/mwcog_sub_links_step6_am_clustering_internal.geojson')
node_file = gpd.read_file(os.getcwd()+'/Network_consolidated/mwcog_sub_nodes_step6_am_clustering_internal.geojson')
link_file.iloc[:, :-1].to_csv("links.csv")
node_file.iloc[:, :-1].to_csv("nodes.csv")
link_data = list(csv.reader(open("links.csv")))
node_data = list(csv.reader(open("nodes.csv")))

In [14]:
Link_Dict = {}
Node_Dict = {}
for i in range(1, len(node_data)):
    Node_Dict[node_data[i][1]] = [node_data[i][2], int(node_data[i][3]), node_data[i][4], node_data[i][5]]
    # list 0 FTYPE
    # list 1 isOD
    # list 2 X
    # list 3 Y
for j in range(1, len(link_data)):
    Link_Dict[link_data[j][1]] = [link_data[j][2], link_data[j][3], float(link_data[j][4]), link_data[j][5],
                                  link_data[j][6], link_data[j][7], int(link_data[j][8])]
    # list 0 A
    # list 1 B
    # list 2 DIST
    # list 3 POSTPD
    # list 4 FACTYPE
    # list 5 LANES
    # list 6 isOD
UpstreamLinkDict = {}
DownstreamLinkDict = {}

# build up empty dict
for i in range(1, len(node_data)):
    UpstreamLinkDict[node_data[i][1]] = []
    DownstreamLinkDict[node_data[i][1]] = []

for i in range(1, len(link_data)):
    UpstreamLinkDict[link_data[i][3]].append(link_data[i][1])
    DownstreamLinkDict[link_data[i][2]].append(link_data[i][1])

## 3.2 Load Moved Link Info (Links not in the consolidated Network)

In [15]:
with open(os.getcwd()+'/Network_consolidated/records_sub_links_step4_am_deletion.json', 'r') as f:
    match_info = json.load(f)

In [16]:
removed_link_list = match_info['delete'] + match_info['combine_delete']

In [17]:
Match_Dict = {}

for key in match_info['combine'].keys():
    if key in Link_Dict.keys():
        for link in match_info['combine'][key]:
            if link != key:
                if link not in Match_Dict.keys():
                    Match_Dict[link] = str()
                    Match_Dict[link] = key
                else:
                    print(link, 'multiple link matched!!')

## 3.3 Move Incidents that cannot be matched to Consoldiated Network

In [18]:
for index, row in Incident_Dataframe_JanFeb.iterrows():
    # print(str(row['LINKID']))
    if str(row['LINKID']) not in list(Link_Dict.keys()): 
        if row['LINKID'] in list(Match_Dict.keys()):
            Incident_Dataframe_JanFeb.loc[index, 'LINKID']= Match_Dict[row['LINKID']]
            print(index, row['LINKID'], Match_Dict[row['LINKID']])
        else:
            Incident_Dataframe_JanFeb.drop(index, inplace=True)

1 56318 31230
13 1480 13596
15 10218 61681
34 66351 37914
157 83132 83131
164 83132 83131
175 63338 7992
196 68581 68580
206 29307 51394
209 83149 83148
211 83128 83126
243 83155 83154
247 16670 51272
277 71803 47291
285 65842 58209


/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772

/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772

344 69015 52136
348 68246 28852
367 38297 58209
372 68290 21178
384 83132 83131
413 29307 51394
431 83132 83131
449 83136 83135
458 68246 28852
484 83132 83131
489 83128 83126
511 25787 47298
527 68293 39728
532 29307 51394
533 69019 65841


/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772

...

11292 39435 34948
11293 39435 34948
11295 39435 34948
11296 39435 34948
11375 38718 52136
11484 83110 25344


/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772

/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772

12082 83160 83159
12086 68246 28852
12122 58506 49569


/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb.drop(index, inplace=True)
/tmp/ipykernel_29949/3562326772

In [19]:
# Consolidated Network Link
print(len(list(set(Incident_Dataframe_JanFeb['LINKID']))))

795


In [20]:
Incident_Dataframe_JanFeb

,Event_ID,Event_Type,Start_Time,Closed_Time,Latitude,Longitude,Road,Direction,Max_Lanes_Closed,A,B,LINKID,start_timestamp,close_timestamp,s_infulence_time,e_infulence_time
0,DDOT_CAPTOP_129937,1 Vehicle Collision,2020-01-04T15:48:44-05:00,2020-01-04T16:33:44-05:00,38.894714,-76.952886,DC 295,North,1.0,20991,20990,23731,1.578171e+09,1.578174e+09,1.578169e+09,1.578177e+09
1,DDOT_CAPTOP_130046,2 Vehicle Collision,2020-01-11T00:43:46-05:00,2020-01-11T01:28:46-05:00,38.823044,-77.016892,I-295,South,NaN,20942,20943,31230,1.578721e+09,1.578724e+09,1.578720e+09,1.578728e+09
2,DDOT_CAPTOP_130282,3+ Vehicle Collision,2020-01-28T06:46:50-05:00,2020-01-28T07:08:11-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.580212e+09,1.580213e+09,1.580210e+09,1.580217e+09
3,DDOT_CAPTOP_130444,2 Vehicle Collision,2020-02-07T08:44:26-05:00,2020-02-07T13:50:00-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.581083e+09,1.581101e+09,1.581081e+09,1.581105e+09
4,DDOT_CAPTOP_130627,2 Vehicle Collision,2020-02-19T23:59:24-05:00,2020-02-20T00:15:00-05:00,38.894714,-76.952886,DC 295,North,NaN,20991,20990,23731,1.582175e+09,1.582176e+09,1.582173e+09,1.582179e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12082,VDOT_INNO2150229274-01112020,Vehicle Fire,2020-01-11T22:15:32-05:00,2020-01-11T23:01:28-05:00,38.901376,-77.190960,I-66,West,2.0,35463,35464,83159,1.578799e+09,1.578802e+09,1.578797e+09,1.578805e+09
12085,VDOT_INNO2202809772-01262020,Vehicle Fire,2020-01-26T23:31:15-05:00,2020-01-27T00:03:52-05:00,38.810716,-77.217544,I-495,South,4.0,35017,35019,45609,1.580099e+09,1.580101e+09,1.580098e+09,1.580105e+09
12086,VDOT_INNO2205061764-01272020,Vehicle Fire,2020-01-27T15:21:20-05:00,2020-01-27T16:19:34-05:00,38.859384,-77.069808,I-395,North,1.0,30250,30199,28852,1.580156e+09,1.580160e+09,1.580155e+09,1.580164e+09
12090,VDOT_INNO2288561089-02212020,Vehicle Fire,2020-02-21T06:09:49-05:00,2020-02-21T09:58:10-05:00,38.873228,-77.044896,VA 90005,South,8.0,30265,30267,40849,1.582283e+09,1.582297e+09,1.582282e+09,1.582301e+09


## 3.4 Move Duplicated Incidents

In [21]:
Incident_Dataframe_JanFeb.drop_duplicates(subset=['Start_Time', 'Closed_Time', 'LINKID', 'Event_Type'], keep='first')

,Event_ID,Event_Type,Start_Time,Closed_Time,Latitude,Longitude,Road,Direction,Max_Lanes_Closed,A,B,LINKID,start_timestamp,close_timestamp,s_infulence_time,e_infulence_time
0,DDOT_CAPTOP_129937,1 Vehicle Collision,2020-01-04T15:48:44-05:00,2020-01-04T16:33:44-05:00,38.894714,-76.952886,DC 295,North,1.0,20991,20990,23731,1.578171e+09,1.578174e+09,1.578169e+09,1.578177e+09
1,DDOT_CAPTOP_130046,2 Vehicle Collision,2020-01-11T00:43:46-05:00,2020-01-11T01:28:46-05:00,38.823044,-77.016892,I-295,South,NaN,20942,20943,31230,1.578721e+09,1.578724e+09,1.578720e+09,1.578728e+09
2,DDOT_CAPTOP_130282,3+ Vehicle Collision,2020-01-28T06:46:50-05:00,2020-01-28T07:08:11-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.580212e+09,1.580213e+09,1.580210e+09,1.580217e+09
3,DDOT_CAPTOP_130444,2 Vehicle Collision,2020-02-07T08:44:26-05:00,2020-02-07T13:50:00-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.581083e+09,1.581101e+09,1.581081e+09,1.581105e+09
4,DDOT_CAPTOP_130627,2 Vehicle Collision,2020-02-19T23:59:24-05:00,2020-02-20T00:15:00-05:00,38.894714,-76.952886,DC 295,North,NaN,20991,20990,23731,1.582175e+09,1.582176e+09,1.582173e+09,1.582179e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12082,VDOT_INNO2150229274-01112020,Vehicle Fire,2020-01-11T22:15:32-05:00,2020-01-11T23:01:28-05:00,38.901376,-77.190960,I-66,West,2.0,35463,35464,83159,1.578799e+09,1.578802e+09,1.578797e+09,1.578805e+09
12085,VDOT_INNO2202809772-01262020,Vehicle Fire,2020-01-26T23:31:15-05:00,2020-01-27T00:03:52-05:00,38.810716,-77.217544,I-495,South,4.0,35017,35019,45609,1.580099e+09,1.580101e+09,1.580098e+09,1.580105e+09
12086,VDOT_INNO2205061764-01272020,Vehicle Fire,2020-01-27T15:21:20-05:00,2020-01-27T16:19:34-05:00,38.859384,-77.069808,I-395,North,1.0,30250,30199,28852,1.580156e+09,1.580160e+09,1.580155e+09,1.580164e+09
12090,VDOT_INNO2288561089-02212020,Vehicle Fire,2020-02-21T06:09:49-05:00,2020-02-21T09:58:10-05:00,38.873228,-77.044896,VA 90005,South,8.0,30265,30267,40849,1.582283e+09,1.582297e+09,1.582282e+09,1.582301e+09


# 4 Filter Weekday Incidents

In [22]:
Incident_Dataframe_JanFeb['Start_Time'] = pd.to_datetime(Incident_Dataframe_JanFeb['Start_Time'])
Incident_Dataframe_JanFeb['Closed_Time'] = pd.to_datetime(Incident_Dataframe_JanFeb['Closed_Time'])

/tmp/ipykernel_29949/3212859686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb['Start_Time'] = pd.to_datetime(Incident_Dataframe_JanFeb['Start_Time'])
/tmp/ipykernel_29949/3212859686.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb['Closed_Time'] = pd.to_datetime(Incident_Dataframe_JanFeb['Closed_Time'])


In [23]:
Incident_Dataframe_JanFeb['Start_Date'] = Incident_Dataframe_JanFeb['Start_Time'].dt.date
Incident_Dataframe_JanFeb['Closed_Date'] = Incident_Dataframe_JanFeb['Closed_Time'].dt.date

/tmp/ipykernel_29949/1899920646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb['Start_Date'] = Incident_Dataframe_JanFeb['Start_Time'].dt.date
/tmp/ipykernel_29949/1899920646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb['Closed_Date'] = Incident_Dataframe_JanFeb['Closed_Time'].dt.date


In [24]:
Incident_Dataframe_JanFeb['Start_Weekday'] = Incident_Dataframe_JanFeb['Start_Time'].dt.dayofweek

/tmp/ipykernel_29949/151373643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb['Start_Weekday'] = Incident_Dataframe_JanFeb['Start_Time'].dt.dayofweek


In [25]:
Incident_Dataframe_JanFeb['Closed_Weekday'] = Incident_Dataframe_JanFeb['Closed_Time'].dt.dayofweek

/tmp/ipykernel_29949/4221619758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Incident_Dataframe_JanFeb['Closed_Weekday'] = Incident_Dataframe_JanFeb['Closed_Time'].dt.dayofweek


In [26]:
Incident_Dataframe_JanFeb = Incident_Dataframe_JanFeb[Incident_Dataframe_JanFeb['Start_Weekday']<5]

In [27]:
Incident_Dataframe_JanFeb

,Event_ID,Event_Type,Start_Time,Closed_Time,Latitude,Longitude,Road,Direction,Max_Lanes_Closed,A,B,LINKID,start_timestamp,close_timestamp,s_infulence_time,e_infulence_time,Start_Date,Closed_Date,Start_Weekday,Closed_Weekday
2,DDOT_CAPTOP_130282,3+ Vehicle Collision,2020-01-28 06:46:50-05:00,2020-01-28 07:08:11-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.580212e+09,1.580213e+09,1.580210e+09,1.580217e+09,2020-01-28,2020-01-28,1,1
3,DDOT_CAPTOP_130444,2 Vehicle Collision,2020-02-07 08:44:26-05:00,2020-02-07 13:50:00-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.581083e+09,1.581101e+09,1.581081e+09,1.581105e+09,2020-02-07,2020-02-07,4,4
4,DDOT_CAPTOP_130627,2 Vehicle Collision,2020-02-19 23:59:24-05:00,2020-02-20 00:15:00-05:00,38.894714,-76.952886,DC 295,North,NaN,20991,20990,23731,1.582175e+09,1.582176e+09,1.582173e+09,1.582179e+09,2020-02-19,2020-02-20,2,3
5,DDOT_CAPTOP_130764,2 Vehicle Collision,2020-02-27 08:26:12-05:00,2020-02-27 09:03:33-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.582810e+09,1.582812e+09,1.582808e+09,1.582816e+09,2020-02-27,2020-02-27,3,3
11,DDOT_CAPTOP_130022,3+ Vehicle Collision,2020-01-10 05:49:16-05:00,2020-01-10 13:20:01-05:00,38.877986,-76.967019,DC 295,North,1.0,20986,21003,56840,1.578653e+09,1.578680e+09,1.578652e+09,1.578684e+09,2020-01-10,2020-01-10,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11865,DDOT_CAPTOP_130209,Disabled Vehicle,2020-01-22 09:13:33-05:00,2020-01-22 13:18:26-05:00,38.900795,-77.014027,3RD STREET TUNNEL,North,1.0,20191,20291,3949,1.579702e+09,1.579717e+09,1.579701e+09,1.579721e+09,2020-01-22,2020-01-22,2,2
11866,DDOT_CAPTOP_130340,Disabled Vehicle,2020-01-31 06:41:14-05:00,2020-01-31 07:06:14-05:00,38.900795,-77.014027,3RD STREET TUNNEL,North,NaN,20191,20291,3949,1.580471e+09,1.580472e+09,1.580469e+09,1.580476e+09,2020-01-31,2020-01-31,4,4
12086,VDOT_INNO2205061764-01272020,Vehicle Fire,2020-01-27 15:21:20-05:00,2020-01-27 16:19:34-05:00,38.859384,-77.069808,I-395,North,1.0,30250,30199,28852,1.580156e+09,1.580160e+09,1.580155e+09,1.580164e+09,2020-01-27,2020-01-27,0,0
12090,VDOT_INNO2288561089-02212020,Vehicle Fire,2020-02-21 06:09:49-05:00,2020-02-21 09:58:10-05:00,38.873228,-77.044896,VA 90005,South,8.0,30265,30267,40849,1.582283e+09,1.582297e+09,1.582282e+09,1.582301e+09,2020-02-21,2020-02-21,4,4


In [37]:
list(Incident_Dataframe_JanFeb['Start_Date'])[0]

datetime.date(2020, 1, 28)

In [39]:
Incident_Dataframe_JanFeb = Incident_Dataframe_JanFeb[~Incident_Dataframe_JanFeb['Start_Date'].isin(federal_holiday_list)]

In [40]:
Incident_Dataframe_JanFeb = Incident_Dataframe_JanFeb[~Incident_Dataframe_JanFeb['Closed_Date'].isin(federal_holiday_list)]

In [41]:
Incident_Dataframe_JanFeb

,Event_ID,Event_Type,Start_Time,Closed_Time,Latitude,Longitude,Road,Direction,Max_Lanes_Closed,A,B,LINKID,start_timestamp,close_timestamp,s_infulence_time,e_infulence_time,Start_Date,Closed_Date,Start_Weekday,Closed_Weekday
2,DDOT_CAPTOP_130282,3+ Vehicle Collision,2020-01-28 06:46:50-05:00,2020-01-28 07:08:11-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.580212e+09,1.580213e+09,1.580210e+09,1.580217e+09,2020-01-28,2020-01-28,1,1
3,DDOT_CAPTOP_130444,2 Vehicle Collision,2020-02-07 08:44:26-05:00,2020-02-07 13:50:00-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.581083e+09,1.581101e+09,1.581081e+09,1.581105e+09,2020-02-07,2020-02-07,4,4
4,DDOT_CAPTOP_130627,2 Vehicle Collision,2020-02-19 23:59:24-05:00,2020-02-20 00:15:00-05:00,38.894714,-76.952886,DC 295,North,NaN,20991,20990,23731,1.582175e+09,1.582176e+09,1.582173e+09,1.582179e+09,2020-02-19,2020-02-20,2,3
5,DDOT_CAPTOP_130764,2 Vehicle Collision,2020-02-27 08:26:12-05:00,2020-02-27 09:03:33-05:00,38.864486,-76.993294,I-295,North,1.0,20982,21422,66943,1.582810e+09,1.582812e+09,1.582808e+09,1.582816e+09,2020-02-27,2020-02-27,3,3
11,DDOT_CAPTOP_130022,3+ Vehicle Collision,2020-01-10 05:49:16-05:00,2020-01-10 13:20:01-05:00,38.877986,-76.967019,DC 295,North,1.0,20986,21003,56840,1.578653e+09,1.578680e+09,1.578652e+09,1.578684e+09,2020-01-10,2020-01-10,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11865,DDOT_CAPTOP_130209,Disabled Vehicle,2020-01-22 09:13:33-05:00,2020-01-22 13:18:26-05:00,38.900795,-77.014027,3RD STREET TUNNEL,North,1.0,20191,20291,3949,1.579702e+09,1.579717e+09,1.579701e+09,1.579721e+09,2020-01-22,2020-01-22,2,2
11866,DDOT_CAPTOP_130340,Disabled Vehicle,2020-01-31 06:41:14-05:00,2020-01-31 07:06:14-05:00,38.900795,-77.014027,3RD STREET TUNNEL,North,NaN,20191,20291,3949,1.580471e+09,1.580472e+09,1.580469e+09,1.580476e+09,2020-01-31,2020-01-31,4,4
12086,VDOT_INNO2205061764-01272020,Vehicle Fire,2020-01-27 15:21:20-05:00,2020-01-27 16:19:34-05:00,38.859384,-77.069808,I-395,North,1.0,30250,30199,28852,1.580156e+09,1.580160e+09,1.580155e+09,1.580164e+09,2020-01-27,2020-01-27,0,0
12090,VDOT_INNO2288561089-02212020,Vehicle Fire,2020-02-21 06:09:49-05:00,2020-02-21 09:58:10-05:00,38.873228,-77.044896,VA 90005,South,8.0,30265,30267,40849,1.582283e+09,1.582297e+09,1.582282e+09,1.582301e+09,2020-02-21,2020-02-21,4,4


In [42]:
Incident_Dataframe_JanFeb.to_csv(os.getcwd()+'/Incidents/JanFeb_selected_inciednts.csv', index=False)